In [1]:
# instalando dependências
! pip install transformers[torch] datasets evaluate scikit-learn matplotlib


You should consider upgrading via the '/home/jonatas/project/material-aula-ds-pucrio/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
# baixando o dataset

from datasets import Features, Value, ClassLabel, load_dataset

dataset = load_dataset("yelp_review_full")

# custom

# class_names = ["0", "1", "2", "3", "4"]
# features = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})
custom_dataset = load_dataset('csv', data_files={'train': 'train_small.csv', 'test': 'test_small.csv'}, 
                              column_names=['label', 'text'], skiprows=[0])
                            #   column_names=['label', 'text'], skiprows=[0], features=features)
custom_dataset = custom_dataset.class_encode_column(column="label")

# criando os splits

small_train_dataset = custom_dataset["train"].shuffle(seed=42).select(range(1000))
small_dev_dataset = custom_dataset["train"].shuffle(seed=42).select(range(1000,1050))
small_test_dataset = custom_dataset["test"].shuffle(seed=42).select(range(500))

/home/jonatas/project/material-aula-ds-pucrio/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset yelp_review_full (/home/jonatas/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 124.56it/s]
Found cached dataset csv (/home/jonatas/.cache/huggingface/datasets/csv/default-7dcc470f94fe408b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 2/2 [00:00<00:00, 971.24it/s]
Loading cached processed dataset at /home/jonatas/.cache/huggingface/datasets/csv/default-7dcc470f94fe408b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-ad4c46bdb608265a.arrow
Loading cached processed dataset at /home/jonatas/.cache/hu

In [3]:
custom_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4274
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 3920
    })
})

In [4]:
custom_dataset["train"][0]

{'label': 0,
 'text': "I got food poisoning after eating at this location, I almost went to the hospital after about 12hrs of vomiting and diarrhea. I will NEVER eat Arby's again"}

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
small_train_dataset[0]

{'label': 3,
 'text': 'I have been going to Jimmy Johns Sandwich shop for 2 years now.I absolutely love there sandwiches and trust me Im a big fan. But today when I came in I was very disappointed. When I received my #9 I had ask the nice guy at the counter for 4 out of the hundereds that were in the giant box of packets of yellow mustard. He said he was only able to give me 2, 2 packets of yellow mustard for the giant sandwich I had in front of me was a joke. He said he would have to charge me 25 cents for a 5 cent packet of mustard. Thats like going to Mcdonalds and ordering 2 large frys and getting two packets of ketchup to fulfill your needs. When I asked Derek why would he have to charge a absurd price he stated \\"Because Jimmy doesnt want you to ruin your sandwich\\". Iam from the greatest country on Earth the United States of America,If I want to I should have the right to put what ever I please on a sandwich. I understand that Derek was doing his job and doing what he was told

In [7]:
set(small_train_dataset["label"])

{0, 1, 2, 3, 4}

In [8]:
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import AutoTokenizer

# carregando o modelo
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

# carregar o bert com pesos aleatórios a partir do config
bert_config = AutoConfig.from_pretrained("bert-base-cased")
bert_config.num_labels = 5
zero_model = AutoModelForSequenceClassification.from_config(bert_config)

# carregando o tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [9]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [10]:
# tokenizando os dados

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_small_dev_dataset = small_dev_dataset.map(tokenize_function, batched=True)
tokenized_small_test_dataset = small_test_dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/jonatas/.cache/huggingface/datasets/csv/default-7dcc470f94fe408b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-7f3c53c4e270cc3f.arrow
Loading cached processed dataset at /home/jonatas/.cache/huggingface/datasets/csv/default-7dcc470f94fe408b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-43c3db75cfad482e.arrow


In [11]:
tokenized_small_train_dataset[0]

{'label': 3,
 'text': 'I have been going to Jimmy Johns Sandwich shop for 2 years now.I absolutely love there sandwiches and trust me Im a big fan. But today when I came in I was very disappointed. When I received my #9 I had ask the nice guy at the counter for 4 out of the hundereds that were in the giant box of packets of yellow mustard. He said he was only able to give me 2, 2 packets of yellow mustard for the giant sandwich I had in front of me was a joke. He said he would have to charge me 25 cents for a 5 cent packet of mustard. Thats like going to Mcdonalds and ordering 2 large frys and getting two packets of ketchup to fulfill your needs. When I asked Derek why would he have to charge a absurd price he stated \\"Because Jimmy doesnt want you to ruin your sandwich\\". Iam from the greatest country on Earth the United States of America,If I want to I should have the right to put what ever I please on a sandwich. I understand that Derek was doing his job and doing what he was told

In [12]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

MODEL_PATH = "test_trainer_v2"

training_args = TrainingArguments(
    output_dir=MODEL_PATH,
    evaluation_strategy="steps",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps=1000,
    learning_rate=3e-5,
    logging_strategy="steps",
    logging_steps=10,
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=2,
    disable_tqdm=True
)

trainer = Trainer(
    # model=model,
    model=zero_model,
    args=training_args,
    train_dataset=tokenized_small_train_dataset,
    eval_dataset=tokenized_small_dev_dataset,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

/home/jonatas/project/material-aula-ds-pucrio/venv/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'loss': 1.7717, 'learning_rate': 2.97e-05, 'epoch': 0.31}
{'loss': 1.6349, 'learning_rate': 2.94e-05, 'epoch': 0.62}


In [ ]:
trainer.train()

In [ ]:
import pandas as pd

df_log = pd.DataFrame(trainer.state.log_history)
df_log

In [ ]:
df_log.plot(y=["loss"])


In [ ]:
df_log["eval_loss"].dropna().plot()

In [ ]:
df_log["eval_accuracy"].dropna().plot()


In [ ]:
# avaliando o modelo

trainer.evaluate(tokenized_small_test_dataset)


In [ ]:
trainer.save_model(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

In [ ]:
# carregando o modelo e rodando um predict

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

text = "This is a really good restaurant"
encoded_input = tokenizer(text, return_tensors="pt")
output = model(**encoded_input)
output.logits

# convertendo a saída para probabilidades

import torch.nn.functional as F

probabilities = F.softmax(output.logits, dim=-1)
probabilities

# convertendo a saída para labels

predicted_class_idx = torch.argmax(probabilities, dim=-1)
predicted_class_idx

probabilities



In [ ]:
encoded_input

In [ ]:
# exemplo de como carregar um dataset customizado

custom_dataset = load_dataset('csv', data_files={'train': 'train_small.csv', 'test': 'test_small.csv'})
custom_dataset

# ClassLabel(names = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'] )